In [ ]:
import os
import sys
import glob

import numpy as np
from scipy.fft import fft, fftfreq
import matplotlib
import matplotlib.pyplot as plt

fontsize = 8
lw = 0.75

matplotlib.rc('font', **{'family': 'Times New Roman', 'size': fontsize})
matplotlib.rc('axes', **{'linewidth': 0.75, 'labelsize': fontsize})
matplotlib.rc('xtick', **{'labelsize': fontsize})
matplotlib.rc('ytick', **{'labelsize': fontsize})
matplotlib.rc('xtick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.minor', **{'width': lw, 'size':1.5})

if '..' not in sys.path:
    sys.path.append('..')
from dlml.data import load_data_generators, load_data_areas, read_area_values

%matplotlib inline

In [ ]:
def plot_data(data_folder, measure_values, var_names,
              gen_id=None, area_id=None, gen_areas_map=None,
              gen_Pnom=None, bus_id=None, line_ids=None, add_omega_ref=True,
              xlim_distr = None, ylim=[-4,4], measure='momentum',
              set_name='training'):
    if gen_id is None and area_id is None:
        raise Exception('One of gen_id or area_id must be not None')
    if gen_id is not None and area_id is not None:
        raise Exception('Either gen_id or area_id must be None')

    measure_values = np.array(measure_values)
    
    if gen_id is not None:
        inertia = {set_name: measure_values}
        time, x, y = load_data_generators([data_folder], [gen_id], inertia, var_names,
                                          max_block_size=500, use_tf=False)
        idx = [np.where(y[set_name] == h)[0] for h in (measure_values.min(), measure_values.max())]
        measure_units = 's'
    else:
        if measure == 'inertia':
            measure_units = 's'
            measure_abbrv = 'H'
        elif measure == 'energy':
            measure_units = r'GW$\cdot$s'
            measure_abbrv = 'E'
        elif measure == 'momentum':
            measure_units = r'GW$\cdot$s$^2$'
            measure_abbrv = 'M'
        data_files = []
        all_data_files = sorted(glob.glob(data_folder + '/*_' + set_name + '_set.h5'))
        H = []
        for data_file in all_data_files:
            _, _, areas_H, _ = read_area_values(data_file, gen_areas_map, gen_Pnom, measure)
            if areas_H[area_id] in measure_values:
                data_files.append(data_file)
            H.append(areas_H[area_id])
        if len(data_files) == 0:
            raise Exception(f'There are no files with the required values of {measure}. ' +
                           f'Accepted values are {H}')
        _, X, _ = load_data_areas({set_name: all_data_files}, var_names,
                                  gen_areas_map, gen_Pnom,
                                  measure, max_block_size=1000,
                                  add_omega_ref=add_omega_ref, use_tf=False)
        x_mean = np.mean(X[set_name], axis=(1,2))
        x_std = np.std(X[set_name], axis=(1,2))
        time, x, y = load_data_areas({set_name: data_files}, var_names,
                                     gen_areas_map, gen_Pnom,
                                     measure, max_block_size=1000,
                                     add_omega_ref = add_omega_ref, use_tf=False)
        idx = [np.where(y[set_name] == h)[0] for h in (measure_values.min(), measure_values.max())]
    n_groups = len(idx)
    
    for i,(m,s) in enumerate(zip(x_mean, x_std)):
        x[set_name][i,:,:] = (x[set_name][i,:,:] - m) / s
    N_vars = len(var_names)

    small = False
    if small:
        fig = plt.figure(figsize=(8.5 / 2.54, N_vars))
        x_offset = [0.13, 0.03]
        y_offset = [0.18, 0.03]
        x_space = 0.1
        y_space = 0.08
        rows = N_vars
        cols = 4
        w = (1 - np.sum(x_offset) - x_space * (cols - 2)) / cols
        h = (1 - np.sum(y_offset) - y_space * (rows - 1)) / rows

        ax = [[plt.axes([x_offset[0], y_offset[0] + (h + y_space) * i, 2*w, h]),
               plt.axes([x_offset[0] + 2 * w + 0.5 * x_space, y_offset[0] + (h + y_space) * i, w, h]),
               plt.axes([x_offset[0] + 3 * w + 2 * x_space, y_offset[0] + (h + y_space) * i, w, h])]
              for i in range(rows-1, -1, -1)]
        for row in ax:
            for a in row:
                for side in 'right','top':
                    a.spines[side].set_visible(False)
        cmap = [[0,0,0], [.75,.75,.75]]
        tmp_ax = []
        for j in range(3):
            for i in range(N_vars):
                tmp_ax.append(ax[i][j])
        ax = tmp_ax
    else:
        fig = plt.figure(figsize=(11, 2 * N_vars))
        gs = fig.add_gridspec(N_vars, 4)
        ax = []
        for i in range(N_vars):
            ax.append(fig.add_subplot(gs[i,:2]))
        for j in (2,3):
            for i in range(N_vars):
                ax.append(fig.add_subplot(gs[i,j]))
        cmap = [[0.2,0.8,0.2], [0.8,0.2,0.8]]

    if xlim_distr is not None:
        lim = xlim_distr
    else:
        lim = 0
    
    # variables used in FFT computation
    N_samples = x[set_name].shape[-1]
    dt = np.diff(time[:2])[0]
    freq = fftfreq(N_samples, dt)[:N_samples//2]

    for i in range(N_vars):
        ### actual traces
        for j in range(n_groups):
            ax[i].plot(time, x[set_name][i, idx[j][:5], :].T, color=cmap[j], lw=0.75)
        ax[i].grid(which='major', axis='y', lw=0.5, ls=':', color=[.6,.6,.6])
        ax[i].set_xticks(np.r_[0 : 61 : 20])
        ax[i].set_yticks(np.r_[-4 : 4.1 : 2])
        if i != N_vars - 1:
            ax[i].set_xticklabels([])

        ### distributions
        for j in range(n_groups):
            n,edges = np.histogram(np.ndarray.flatten(x[set_name][i,:,:][idx[j],:]), \
                                   bins=100, range=ylim, density=True)
            lim = n.max() if n.max() > lim else lim
            ax[i+N_vars].plot(n, edges[:-1], color=cmap[j], linewidth=1.2,
                              label=f'{measure_abbrv}={measure_values[j]:.3f} {measure_units}')
        ax[i+N_vars].set_yticklabels([])
        ax[i+N_vars].set_xticks([0, 0.5])
        ax[i+N_vars].set_yticks(np.r_[-4 : 4.1 : 2])
        if i != N_vars - 1:
            ax[i+N_vars].set_xticklabels([])
        ax[i+N_vars].grid(which='major', axis='y', lw=0.5, ls=':', color=[.6,.6,.6])
        
        ### FFTs
        for j in range(n_groups):
            Xf = fft(x[set_name][i, idx[j], :])
            Xf = 2.0 / N_samples * np.abs(Xf[:, 0 : N_samples // 2])
            m = Xf.mean(axis=0)
            s = Xf.std(axis=0)
            ci = 1.96 * s / np.sqrt(Xf.shape[0])
            ax[i+2*N_vars].fill_between(freq, 20*np.log10(m-ci), 20*np.log10(m+ci), color=cmap[j], lw=1)
        ax[i+2*N_vars].set_xscale('log')
        if i != N_vars - 1:
            ax[i+2*N_vars].set_xticklabels([])
       
    for a in ax:
        for side in 'right', 'top':
            a.spines[side].set_visible(False)
    for i,var_name in enumerate(var_names):
        ax[i].set_ylabel('Norm. ' + var_name)

    for i in range(N_vars):
        ax[i].set_ylim(ylim)
        ax[N_vars + 1].set_ylim(ylim)
        ax[N_vars + i].plot([0, lim*1.05], [0,0], '--', lw=1, color=[.6,.6,.6])
        ax[2*N_vars + i].set_ylabel('Power [dB]')
    ax[-1].set_xlabel('Frequency [Hz]')
    
    ax[N_vars - 1].set_xlabel('Time [s]')
    ax[2 * N_vars - 1].set_xlabel('Fraction')
    if not small:
        ax[2*N_vars-1].legend(loc='lower right', frameon=False)
        fig.tight_layout(pad=0.1)

    if gen_id is not None:
        fig.savefig(data_folder + f'/generator_{gen_id}.pdf');
    elif bus_id is not None:
        fig.savefig(data_folder + f'/bus_{bus_id}.pdf');
    elif line_ids is not None:
        fig.savefig(data_folder + f'/line_{line_ids[0]}_{line_ids[1]}.pdf');

## Two area network

## IEEE39 Hiskens' style

In [ ]:
bus_IDs = 3,# 14, 17, 39
area = 1
H_G1 = 500
data_folder = f'../data/IEEE39/Hiskens/var_H_area_{area}/IEEE39_D=2_DZA=0.0/H_G1_500/stoch_load_bus_3'
gen_areas_map = [['Pg31','Pg32','Pg39'], ['Pg33','Pg34','Pg35','Pg36'], ['Pg37','Pg38'], ['Pg30']]
gen_Pnom = {gen: 100e6 for area in gen_areas_map for gen in area}
measure = 'momentum'
print(' ' * 58 + '{:^14s}'.format(measure.upper()))
print('=' * 77)
print('{:^56s} {:^6s} {:^6s} {:^6s}'.format('Filename', 'A1','A2','A3'))
print('=' * 77)
for data_file in sorted(glob.glob(data_folder + '/*_training_set.h5')):
    gen_IDs, gen_H, area_meas, _ = read_area_values(data_file, gen_areas_map, gen_Pnom, measure)
    a,b = os.path.basename(data_file)[:30], os.path.basename(data_file)[-23:]
    print(f'{a}...{b} {area_meas[0]:6.3f} {area_meas[1]:6.3f} {area_meas[2]:6.3f}')

In [ ]:
if measure == 'momentum':
    if area in (1,2):
        measure_values = [8/30, 14/30]
    elif area == 3:
        measure_values = [0.1, 0.3]
else:
    raise Exception(f'Measure {measure} not valid')
# var_names_fmt = 'omegael_bus{}','Pe_bus{}','Qe_bus{}','Vd_bus{}','Vq_bus{}'
# var_names_fmt = 'Pe_bus{}','Qe_bus{}','Vd_bus{}','Vq_bus{}'
var_names_fmt = 'Vd_bus{}', 'Pe_bus{}'
for bus in bus_IDs:
    var_names = tuple(var_name.format(bus) for var_name in var_names_fmt)
    plot_data(data_folder, measure_values, var_names, bus_id=bus, area_id=area-1,
              gen_areas_map=gen_areas_map, gen_Pnom=gen_Pnom, xlim_distr = 0.5, ylim=[-4,4])

## IEEE39 with DIgSILENT PowerFactory

In [ ]:
area = 1
data_folder = f'../data/IEEE39/PowerFactory/var_H_area_{area}'
gen_areas_map = [['G 02','G 03','G 10'], ['G 04','G 05','G 06','G 07'], ['G 08','G 09'], ['G 01']]
gen_Pnom = {'G 01': 10000e6, 'G 02': 700e6, 'G 03': 800e6, 'G 04': 800e6, 'G 05': 300e6,
            'G 06': 800e6, 'G 07': 700e6, 'G 08': 700e6, 'G 09': 1000e6, 'G 10': 1000e6}
measure = 'inertia'
set_name = 'test'
print(' ' * 64 + '{:^14s}'.format(measure.upper()))
print('=' * 84)
print('{:^56s} {:^6s} {:^6s} {:^6s} {:^6s}'.format('Filename', 'A1', 'A2', 'A3', 'A4'))
print('=' * 84)
for data_file in sorted(glob.glob(data_folder + '/*_' + set_name + '_set.h5')):
    gen_IDs, gen_H, area_meas, _ = read_area_values(data_file, gen_areas_map, gen_Pnom, measure)
    a,b = os.path.basename(data_file)[:30], os.path.basename(data_file)[-23:]
    print(f'{a}...{b} {area_meas[0]:6.3f} {area_meas[1]:6.3f} {area_meas[2]:6.3f} {area_meas[3]:6.3f}')

In [ ]:
if set_name == 'training':
    measure_values = [3.00004, 5.1002]
elif set_name == 'test':
    measure_values = [3.09984, 5.2]
else:
    raise Exception(f'set name {set_name} is not valid')
# var_names_fmt = 'omegael_bus{}','Pe_bus{}','Qe_bus{}','Vd_bus{}','Vq_bus{}'
var_names_fmt = 'Pe_bus{}','Qe_bus{}','Vd_bus{}','Vq_bus{}'
bus_IDs = 3,
for bus in bus_IDs:
    var_names = tuple(var_name.format(bus) for var_name in var_names_fmt)
    plot_data(data_folder, measure_values, var_names, bus_id=bus, area_id=area-1,
              gen_areas_map=gen_areas_map, gen_Pnom=gen_Pnom, xlim_distr = 0.5, ylim=[-4,4],
              measure='inertia', add_omega_ref=False, set_name=set_name)

## IEEE39 converted from PowerFactory

In [ ]:
area = 1
data_folder = f'../data/IEEE39/converted_from_PowerFactory/all_stoch_loads/var_H_area_{area}'
gen_areas_map = [['G02','G03'], ['G04','G05','G06','G07'], ['G08','G09', 'G10'], ['G01']]
gen_Pnom = {'G01': 10000e6, 'G02': 700e6, 'G03': 800e6, 'G04': 800e6, 'G05': 300e6,
            'G06': 800e6, 'G07': 700e6, 'G08': 700e6, 'G09': 1000e6, 'G10': 1000e6}
measure = 'momentum'
set_name = 'test'
print(' ' * 64 + '{:^14s}'.format(measure.upper()))
print('=' * 84)
print('{:^56s} {:^6s} {:^6s} {:^6s} {:^6s}'.format('Filename', 'A1', 'A2', 'A3', 'A4'))
print('=' * 84)
for data_file in sorted(glob.glob(data_folder + '/*_' + set_name + '_set.h5')):
    gen_IDs, gen_H, area_meas, _ = read_area_values(data_file, gen_areas_map, gen_Pnom, measure)
    a,b = os.path.basename(data_file)[:30], os.path.basename(data_file)[-23:]
    print(f'{a}...{b} {area_meas[0]:6.3f} {area_meas[1]:6.3f} {area_meas[2]:6.3f} {area_meas[3]:6.3f}')

In [ ]:
if set_name == 'training':
    measure_values = [0.14999666666666667, 0.30000333333333334]
elif set_name == 'test':
    measure_values = [0.155, 0.30498]
else:
    raise Exception(f'set name {set_name} is not valid')
var_names_fmt = 'Vd_bus{}','Vq_bus{}'
bus_IDs = 3, 14, 17, 39
for bus in bus_IDs:
    var_names = tuple(var_name.format(bus) for var_name in var_names_fmt)
    plot_data(data_folder, measure_values, var_names, bus_id=bus, area_id=area-1,
              gen_areas_map=gen_areas_map, gen_Pnom=gen_Pnom, xlim_distr = 0.5, ylim=[-4,4],
              measure=measure, add_omega_ref=False, set_name=set_name)

In [ ]:
if set_name == 'training':
    measure_values = [0.14999666666666667, 0.30000333333333334]
elif set_name == 'test':
    measure_values = [0.155, 0.30498]
else:
    raise Exception(f'set name {set_name} is not valid')
var_names_fmt = 'Id_line_{}_{}','Iq_line_{}_{}'
lines_IDs = (3,4), (14,15), (16,17), (1,39)
for lines in lines_IDs:
    var_names = tuple(var_name.format(lines[0], lines[1]) for var_name in var_names_fmt)
    plot_data(data_folder, measure_values, var_names, line_ids=lines, area_id=area-1,
              gen_areas_map=gen_areas_map, gen_Pnom=gen_Pnom, xlim_distr = 0.5, ylim=[-4,4],
              measure=measure, add_omega_ref=False, set_name=set_name)

## IEEE39 converted from PowerFactory (with compensators)

In [ ]:
bus_IDs = 3, 14, 17, 39
area = 1
data_folder = f'../data/IEEE39/converted_from_PowerFactory/all_stoch_loads/var_H_area_{area}_comp'
gen_areas_map = [['G02','G03','Comp11'],
                 ['G04','G05','G06','G07','Comp21'],
                 ['G08','G09', 'G10','Comp31'],
                 ['G01']]
gen_Pnom = {'G01': 10000e6, 'G02': 700e6, 'G03': 800e6, 'G04': 800e6, 'G05': 300e6,
            'G06': 800e6, 'G07': 700e6, 'G08': 700e6, 'G09': 1000e6, 'G10': 1000e6,
            'Comp11': 100e6, 'Comp21': 100e6, 'Comp31': 100e6}
measure = 'momentum'
set_name = 'test'
print(' ' * 64 + '{:^14s}'.format(measure.upper()))
print('=' * 84)
print('{:^56s} {:^6s} {:^6s} {:^6s} {:^6s}'.format('Filename', 'A1', 'A2', 'A3', 'A4'))
print('=' * 84)
for data_file in sorted(glob.glob(data_folder + '/*_' + set_name + '_set.h5')):
    gen_IDs, gen_H, area_meas, _ = read_area_values(data_file, gen_areas_map, gen_Pnom, measure)
    a,b = os.path.basename(data_file)[:30], os.path.basename(data_file)[-23:]
    print(f'{a}...{b} {area_meas[0]:6.3f} {area_meas[1]:6.3f} {area_meas[2]:6.3f} {area_meas[3]:6.3f}')

In [ ]:
if set_name == 'training':
    measure = []
elif set_name == 'test':
    measure_values = [0.15500000003333333, 0.30500000013333334]
else:
    raise Exception(f'set name {set_name} is not valid')
var_names_fmt = 'Vd_bus{}','Vq_bus{}'
for bus in bus_IDs:
    var_names = tuple(var_name.format(bus) for var_name in var_names_fmt)
    plot_data(data_folder, measure_values, var_names, bus_id=bus, area_id=area-1,
              gen_areas_map=gen_areas_map, gen_Pnom=gen_Pnom, xlim_distr = 0.5, ylim=[-4,4],
              measure=measure, add_omega_ref=False, set_name=set_name)

In [ ]:
if set_name == 'training':
    measure = []
elif set_name == 'test':
    measure_values = [0.15500000003333333, 0.30500000013333334]
else:
    raise Exception(f'set name {set_name} is not valid')
var_names_fmt = 'Id_line_{}_{}','Iq_line_{}_{}'
lines_IDs = (3,4), (14,15), (16,17), (1,39)
for lines in lines_IDs:
    var_names = tuple(var_name.format(lines[0], lines[1]) for var_name in var_names_fmt)
    plot_data(data_folder, measure_values, var_names, line_ids=lines, area_id=area-1,
              gen_areas_map=gen_areas_map, gen_Pnom=gen_Pnom, xlim_distr = 0.5, ylim=[-4,4],
              measure=measure, add_omega_ref=False, set_name=set_name)